In [1]:
import numpy as np
import pandas as pd
import os

import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.io as pio
from plotly.subplots import make_subplots
# setting default template to plotly_white for all visualizations
pio.templates.default = "plotly_white"
%matplotlib inline
import gc

import utils_eda as uteda

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
import os
import gc
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm
from shutil import copyfile
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [2]:
# Install W&B 
!pip install -q --upgrade wandb
# Login 
import os
key = os.getenv('WANDB_API_KEY')
# print(key)
import wandb
wandb.login(key=key)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)


True

#  Load Data

In [23]:
TRAIN_PATH = 'input/siim-covid19-resized-to-512px-png/train/'
IMG_SIZE = 512
BATCH_SIZE = 64
EPOCHS = 300
HOME= '~/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection'
CLASS_NUMBER=1

In [4]:
PATH = './input/siim-covid19-detection'
# submission = pd.read_csv(os.path.join(PATH,'sample_submission.csv'), index_col=None)
df = pd.read_csv(os.path.join(PATH,'train_image_level.csv'), index_col=None)
# Modify values in the id column
df['id'] = df.apply(lambda row: row.id.split('_')[0], axis=1)
# Add absolute path
df['path'] = df.apply(lambda row: TRAIN_PATH+row.id+'.jpg', axis=1)
# Get image level labels
df['image_level'] = df.apply(lambda row: row.label.split(' ')[0], axis=1)

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)
print(f"Train image level csv shape : {df.shape}\nTrain study level csv shape : {df.shape}")

Train image level csv shape : (6334, 6)
Train study level csv shape : (6334, 6)


In [5]:
# Load meta.csv file
# Original dimensions are required to scale the bounding box coordinates appropriately.
meta_df = pd.read_csv('input/siim-covid19-resized-to-512px-png/meta.csv')
train_meta_df = meta_df.loc[meta_df.split == 'train']
train_meta_df = train_meta_df.drop('split', axis=1)
train_meta_df.columns = ['id', 'dim0', 'dim1']

train_meta_df.head(10)

# Merge both the dataframes
df = df.merge(train_meta_df, on='id',how="left")
df.head(2)

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 1026.65662, 'height': 1917.30292}, {'x': 2245.91208, 'y': 591.20528, 'width': 1094.66162, 'height': 1761.54944}]",opacity 1 789.28836 582.43035 1815.94498 2499.73327 opacity 1 2245.91208 591.20528 3340.5737 2352.75472,5776db0cec75,input/siim-covid19-resized-to-512px-png/train/000a312787f2.jpg,opacity,3488,4256
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,input/siim-covid19-resized-to-512px-png/train/000c3a3f293f.jpg,none,2320,2832


# Make small training set

# Find the bounding box distribution

In [6]:
#lets find max number bounding boxes in train set bounding boxes
f=lambda x:x.count('{') if type(x) is str else 0
numb_bboxes_per_row=list(map(f,df.boxes))
m=max(numb_bboxes_per_row)
print(f'Maximum number of bounding boxes={m}\n')
tots=0

percentages=[]
for i in range(0, m+1):
    tot = numb_bboxes_per_row.count(i)
    print(f'number images with {i} bounding boxes={tot}')
    percentages.append(tot)
    tots+=tot
print(f'\nTotal images={tots}, total with bounding boxes={tots-numb_bboxes_per_row.count(0)}\n')

percentages=[x/tots for x in percentages]
for i,pct in enumerate(percentages):
#     "{:.2f}".format(float)
    print(f'{"{:.2f}".format(pct)} % has {i} bounding boxes')
# numb_bboxes_per_row

Maximum number of bounding boxes=8

number images with 0 bounding boxes=2040
number images with 1 bounding boxes=973
number images with 2 bounding boxes=3113
number images with 3 bounding boxes=183
number images with 4 bounding boxes=23
number images with 5 bounding boxes=1
number images with 6 bounding boxes=0
number images with 7 bounding boxes=0
number images with 8 bounding boxes=1

Total images=6334, total with bounding boxes=4294

0.32 % has 0 bounding boxes
0.15 % has 1 bounding boxes
0.49 % has 2 bounding boxes
0.03 % has 3 bounding boxes
0.00 % has 4 bounding boxes
0.00 % has 5 bounding boxes
0.00 % has 6 bounding boxes
0.00 % has 7 bounding boxes
0.00 % has 8 bounding boxes


# Create a smaller dataframe with images with diff number bounding boxes (not representative of the above proportions though)

In [7]:
NUMB_BOXES='0_thru_4'

In [8]:
NUMB_ROWS_OF_EACH_BB_NUMBER = 100
import math
# numb_entries=[math.floor(x*NUMB_ROWS_WANTED) for x in percentages]
# print(f'number entries={numb_entries}')

tmp_df=df.copy()

#add bboxes column
tmp_df['n_bboxes']=numb_bboxes_per_row

# start with empty dataframe
df1 = pd.DataFrame()
  
#just want those with up to 4 bounding boxes 
for i in range(5):
    numb=min(numb_bboxes_per_row.count(i),NUMB_ROWS_OF_EACH_BB_NUMBER)
    df1=df1.append(tmp_df[tmp_df['n_bboxes']==i][:numb])

    #reindex start from 0, get rid of old non sequential index 
df1.reset_index(inplace=True, drop=True)
df1.head()

# len(df[df['n_bboxes']==4])
len(df1)
df1

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1,n_bboxes
0,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,input/siim-covid19-resized-to-512px-png/train/000c3a3f293f.jpg,none,2320,2832,0
1,0023f02ae886,NaN,none 1 0 0 1 1,2fa400b873f5,input/siim-covid19-resized-to-512px-png/train/0023f02ae886.jpg,none,2330,2846,0
2,00c1515729a8,NaN,none 1 0 0 1 1,1a58b43cf286,input/siim-covid19-resized-to-512px-png/train/00c1515729a8.jpg,none,2539,3050,0
3,00e3a7e91a34,NaN,none 1 0 0 1 1,74ba8f2badcb,input/siim-covid19-resized-to-512px-png/train/00e3a7e91a34.jpg,none,1760,2140,0
4,010214f9b600,NaN,none 1 0 0 1 1,2a7b4e669dda,input/siim-covid19-resized-to-512px-png/train/010214f9b600.jpg,none,2539,3050,0
...,...,...,...,...,...,...,...,...,...
418,98721e6d2bde,"[{'x': 1029.49078, 'y': 2027.81169, 'width': 654.41504, 'height': 468.37708}, {'x': 1143.30204, 'y': 1594.4528, 'width': 520.90576, 'height': 391.77368}, {'x': 961.64159, 'y': 898.45281, 'width': 759.47198, 'height': 625.96234}, {'x': 3045.26417, 'y': 2194.15094, 'width': 687.24585, 'height': 577.81152}]",opacity 1 1029.49078 2027.81169 1683.90582 2496.1887699999997 opacity 1 1143.30204 1594.4528 1664.2078000000001 1986.22648 opacity 1 961.64159 898.45281 1721.11357 1524.41515 opacity 1 3045.26417 2194.15094 3732.5100199999997 2771.9624599999997,63e24b10c016,input/siim-covid19-resized-to-512px-png/train/98721e6d2bde.jpg,opacity,3480,4240,4
419,a5c4c8a44bf6,"[{'x': 1630.50333, 'y': 658.36541, 'width': 428.3175, 'height': 602.1275}, {'x': 473.83918, 'y': 410.06542, 'width': 585.57416, 'height': 515.22247}, {'x': 264.85333, 'y': 995.63956, 'width': 492.46167, 'height': 569.02081}, {'x': 1901.56419, 'y': 1695.01784, 'width': 208.98572, 'height': 215.19336}]",opacity 1 1630.50333 658.36541 2058.82083 1260.49291 opacity 1 473.83918 410.06542 1059.41334 925.2878900000001 opacity 1 264.85333 995.63956 757.315 1564.66037 opacity 1 1901.56419 1695.01784 2110.54991 1910.2112,547aae81c04c,input/siim-covid19-resized-to-512px-png/train/a5c4c8a44bf6.jpg,opacity,2312,2483,4
420,f5983f676443,"[{'x': 1285.34689, 'y': 847.33862, 'width': 721.22314, 'height': 725.16418}, {'x': 2916.96637, 'y': 1056.21744, 'width': 788.22192, 'height': 807.92761}, {'x': 966.11699, 'y': 1745.91166, 'width': 595.10754, 'height': 736.98755}, {'x': 3366.25289, 'y': 2124.25821, 'width': 555.69629, 'height': 555.69629}]",opacity 1 1285.34689 847.33862 2006.5700299999999 1572.5028 opacity 1 2916.96637 1056.21744 3705.18829 1864.1450499999999 opacity 1 966.11699 1745.91166 1561.22453 2482.89921 opacity 1 3366.25289 2124.25821 3921.94918 2679.9545,c74ee3da0583,input/siim-covid19-resized-to-512px-png/train/f5983f676443.jpg,opacity,3480,4240,4
421,f774a9c27944,"[{'x': 1667.19025, 'y': 175.2, 'width': 498.34667, 'height': 759.2}, {'x': 588.73692, 'y': 241.38667, 'width': 611.25333, 'height': 595.68}, {'x': 452.47025, 'y': 969.44, 'width': 537.28, 'height': 517.81333}, {'x': 1577.64359, 'y': 1051.2, 'width': 634.61333, 'height': 580.10667}]",opacity 1 1667.19025 175.2 2165.53692 934.4000000000001 opacity 1 588.73692 241.38667 1199.99025 837.0666699999999 opacity 1 452.47025 969.44 989.75025 1487.25333 opacity 1 1577.64359 1051.2 2212.25692 1631.30667,7fc0becb14fa,input/siim-covid19-resized-to-512px-png/train/f774a9c27944.jpg,opacity,2336,2836,4


## 🍘 Train-validation split  THERE IS NONE! I AM TRAINING ON TRAING DATA AND TESTING ON SAME DATA TO VERIFY FIT!

In [14]:

# Create train and validation split.
TEST_SIZE=0.3
train_df, valid_df = train_test_split(df1, test_size=0.2, random_state=42, stratify=df1.image_level.values)

train_df.loc[:, 'split'] = 'train'
valid_df.loc[:, 'split'] = 'valid'

df = pd.concat([train_df, valid_df]).reset_index(drop=True)
print(f'Size of dataset: {len(df)}, training images: {len(train_df)}. validation images: {len(valid_df)}')

Size of dataset: 423, training images: 338. validation images: 85


In [10]:
df

,id,boxes,label,StudyInstanceUID,path,image_level,dim0,dim1
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 1026.65662, 'height': 1917.30292}, {'x': 2245.91208, 'y': 591.20528, 'width': 1094.66162, 'height': 1761.54944}]",opacity 1 789.28836 582.43035 1815.94498 2499.73327 opacity 1 2245.91208 591.20528 3340.5737 2352.75472,5776db0cec75,input/siim-covid19-resized-to-512px-png/train/000a312787f2.jpg,opacity,3488,4256
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,input/siim-covid19-resized-to-512px-png/train/000c3a3f293f.jpg,none,2320,2832
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867.79767, 'height': 999.78214}, {'x': 1792.69064, 'y': 402.5525, 'width': 617.02734, 'height': 1204.358}]",opacity 1 677.42216 197.97662 1545.21983 1197.75876 opacity 1 1792.69064 402.5525 2409.71798 1606.9105,9d514ce429a7,input/siim-covid19-resized-to-512px-png/train/0012ff7358bc.jpg,opacity,2544,3056
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.00012, 'height': 604}]",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,input/siim-covid19-resized-to-512px-png/train/001398f4ff4f.jpg,opacity,3520,4280
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'height': 1106}, {'x': 2578.56661, 'y': 998.66667, 'width': 662.66667, 'height': 1120}]",opacity 1 623.23328 1050 1337.23328 2156 opacity 1 2578.56661 998.66667 3241.23328 2118.66667,dfd9fdd85a3e,input/siim-covid19-resized-to-512px-png/train/001bd15d1891.jpg,opacity,2800,3408
...,...,...,...,...,...,...,...,...
6329,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,input/siim-covid19-resized-to-512px-png/train/ffcc6edd9445.jpg,none,4240,3480
6330,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,input/siim-covid19-resized-to-512px-png/train/ffd91a2c4ca0.jpg,none,2800,3408
6331,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 316.41699, 'height': 451.63758}, {'x': 2375.87717, 'y': 1830.89015, 'width': 267.7373, 'height': 305.59912}, {'x': 707.25199, 'y': 722.07926, 'width': 392.14044, 'height': 849.18683}]",opacity 1 2197.38566 841.07361 2513.80265 1292.71119 opacity 1 2375.87717 1830.89015 2643.6144700000004 2136.48927 opacity 1 707.25199 722.07926 1099.3924299999999 1571.26609,7eed9af03814,input/siim-covid19-resized-to-512px-png/train/ffd9b6cf2961.jpg,opacity,2388,3050
6332,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 1496.25016, 'height': 2604.58334}, {'x': 1005.8125, 'y': 1584.67711, 'width': 662.22913, 'height': 775.83337}]",opacity 1 2729.27083 332.26044 4225.52099 2936.84378 opacity 1 1005.8125 1584.67711 1668.0416300000002 2360.51048,a0cb0b96fb3d,input/siim-covid19-resized-to-512px-png/train/ffdc682f7680.jpg,opacity,3488,4256


## 🍚 Prepare Required Folder Structure

The required folder structure for the dataset directory is: 

```
/parent_folder
    /dataset
         /images
             /train
             /val
         /labels
             /train
             /val
    /yolov5
```

Note that I have named the directory `covid`.

In [15]:
os.makedirs('tmp/covid_small/images/train', exist_ok=True)
os.makedirs('tmp/covid_small/images/valid', exist_ok=True)

os.makedirs('tmp/covid_small/labels/train', exist_ok=True)
os.makedirs('tmp/covid_small/labels/valid', exist_ok=True)

In [16]:
# # Move the images to relevant split folder.
# for i in tqdm(range(len(df))):
#     row = df.loc[i]
#     if row.split == 'train':
# #         print(row.path)
#         copyfile(row.path, f'tmp/covid_small/images/train/{row.id}.jpg')
#     else:
#         copyfile(row.path, f'tmp/covid_small/images/valid/{row.id}.jpg')

# TRAIN AND TEST ON SAME DATA
for i in tqdm(range(len(df))):
    row = df.loc[i]
    copyfile(row.path, f'tmp/covid_small/images/train/{row.id}.jpg')
    copyfile(row.path, f'tmp/covid_small/images/valid/{row.id}.jpg')

100%|██████████| 423/423 [00:00<00:00, 4517.03it/s]


## 🍜 Create `.YAML` file

The `data.yaml`, is the dataset configuration file that defines 

1. an "optional" download command/URL for auto-downloading, 
2. a path to a directory of training images (or path to a *.txt file with a list of training images), 
3. a path to a directory of validation images (or path to a *.txt file with a list of validation images), 
4. the number of classes, 
5. a list of class names.

> 📍 Important: In this competition, each image can either belong to `opacity` or `none` image-level labels. That's why I have  used the number of classes, `nc` to be 2. YOLOv5 automatically handles the images without any bounding box coordinates. 

> 📍 Note: The `data.yaml` is created in the `yolov5/data` directory as required. 

In [ ]:
%pwd

## 🍮 Prepare Bounding Box Coordinated for YOLOv5

For every image with **bounding box(es)** a `.txt` file with the same name as the image will be created in the format shown below:

* One row per object. <br>
* Each row is class `x_center y_center width height format`. <br>
* Box coordinates must be in normalized xywh format (from 0 - 1). We can normalize by the boxes in pixels by dividing `x_center` and `width` by image width, and `y_center` and `height` by image height. <br>
* Class numbers are zero-indexed (start from 0). <br>

> 📍 Note: We don't have to remove the images without bounding boxes from the training or validation sets. 

In [17]:
# Get the raw bounding box by parsing the row value of the label column.
# Ref: https://www.kaggle.com/yujiariyasu/plot-3positive-classes
def get_bbox(row):
    bboxes = []
    bbox = []
    for i, l in enumerate(row.label.split(' ')):
        if (i % 6 == 0) | (i % 6 == 1):
            continue
        bbox.append(float(l))
        if i % 6 == 5:
            bboxes.append(bbox)
            bbox = []  
            
    return bboxes

# Scale the bounding boxes according to the size of the resized image. 
def scale_bbox(row, bboxes):
    # Get scaling factor
    scale_x = IMG_SIZE/row.dim1
    scale_y = IMG_SIZE/row.dim0
    
    scaled_bboxes = []
    for bbox in bboxes:
        x = int(np.round(bbox[0]*scale_x, 4))
        y = int(np.round(bbox[1]*scale_y, 4))
        x1 = int(np.round(bbox[2]*(scale_x), 4))
        y1= int(np.round(bbox[3]*scale_y, 4))

        scaled_bboxes.append([x, y, x1, y1]) # xmin, ymin, xmax, ymax
        
    return scaled_bboxes

# Convert the bounding boxes in YOLO format.
def get_yolo_format_bbox(img_w, img_h, bboxes):
    yolo_boxes = []
    for bbox in bboxes:
        w = bbox[2] - bbox[0] # xmax - xmin
        h = bbox[3] - bbox[1] # ymax - ymin
        xc = bbox[0] + int(np.round(w/2)) # xmin + width/2
        yc = bbox[1] + int(np.round(h/2)) # ymin + height/2
        
        yolo_boxes.append([xc/img_w, yc/img_h, w/img_w, h/img_h]) # x_center y_center width height
    
    return yolo_boxes

In [ ]:
df

In [ ]:
a=df[(df['split']=='valid')]
len(a)

In [18]:
# # Prepare the txt files for bounding box
# # for i in tqdm(range(10)):
# for i in tqdm(range(len(df))):
#     row = df.loc[i]
#     # Get image id
#     img_id = row.id
#     # Get split
#     split = row.split
#     # Get image-level label
#     label = row.image_level
    
#     if row.split=='train':
#         file_name = f'tmp/covid_small/labels/train/{row.id}.txt'
#     else:
#         file_name = f'tmp/covid_small/labels/valid/{row.id}.txt'
        
    
#     if label=='opacity':
#         # Get bboxes
#         bboxes = get_bbox(row)
#         # Scale bounding boxes
#         scale_bboxes = scale_bbox(row, bboxes)
#         # Format for YOLOv5
#         yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)
        
#         with open(file_name, 'w') as f:
#             for bbox in yolo_bboxes:
#                 bbox = [CLASS_NUMBER]+bbox
#                 bbox = [str(i) for i in bbox]
#                 bbox = ' '.join(bbox)
# #                 print(bbox)
#                 f.write(bbox)
#                 f.write('\n')
# Prepare the txt files for bounding box
# for i in tqdm(range(10)):
for i in tqdm(range(len(df))):
    row = df.loc[i]
    # Get image id
    img_id = row.id
    # Get split
    split = row.split
    # Get image-level label
    label = row.image_level
  
    fls=[f'tmp/covid_small/labels/train/{row.id}.txt',f'tmp/covid_small/labels/valid/{row.id}.txt' ]
 
    for file_name in fls: 
        if label=='opacity':
            # Get bboxes
            bboxes = get_bbox(row)
            # Scale bounding boxes
            scale_bboxes = scale_bbox(row, bboxes)
            # Format for YOLOv5
            yolo_bboxes = get_yolo_format_bbox(IMG_SIZE, IMG_SIZE, scale_bboxes)

            with open(file_name, 'w') as f:
                for bbox in yolo_bboxes:
                    bbox = [CLASS_NUMBER]+bbox
                    bbox = [str(i) for i in bbox]
                    bbox = ' '.join(bbox)
    #                 print(bbox)
                    f.write(bbox)
                    f.write('\n')

100%|██████████| 423/423 [00:00<00:00, 2998.91it/s]


# 🚅 Train with W&B



In [19]:
# Create .yaml file 
%cd {HOME}
import yaml


data_yaml = dict(
    train = '../covid_small/images/train',
    val = '../covid_small/images/valid',
    
    #train with 1 or 2 classes?
#     nc = 1,
#     names = ['opacity']
    nc = 2,
    names = ['opacity','none']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('tmp/yolov5/data/data1.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat tmp/yolov5/data/data1.yaml

/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection
{names: [opacity, none], nc: 2, train: ../covid_small/images/train, val: ../covid_small/images/valid}


```
--img {IMG_SIZE} \ # Input image size.
--batch {BATCH_SIZE} \ # Batch size
--epochs {EPOCHS} \ # Number of epochs
--data data.yaml \ # Configuration file
--weights yolov5l.pt \ # Model name
--save_period 1\ # Save model after interval
--project kaggle-siim-covid # W&B project name
```

In [20]:
%cd tmp/yolov5/
    #train with 1 or 2 classes?
# !python train.py --img {IMG_SIZE} \
#                  --batch {BATCH_SIZE} \
#                  --epochs {EPOCHS} \
#                  --data data1.yaml \
#                  --weights yolov5s.pt \
#                  --save_period 1\
#                  --project kaggle-siim-covid_tiny\
#                  --single-cls    
    
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data1.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project kaggle-siim-covid_tiny
%cd ..

/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection/tmp/yolov5
train: weights=yolov5s.pt, cfg=, data=data1.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=100, batch_size=64, img_size=[512], rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=kaggle-siim-covid_tiny, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1
github: Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds, for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v5.0-294-gdd62e2d torch 1.9.0+cu102 CUDA:0 (TITAN Xp, 12194.0625MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5

# Now a different yaml file (1 class)

In [24]:
# Create .yaml file 
%cd {HOME}
import yaml


data_yaml = dict(
    train = '../covid_small/images/train',
    val = '../covid_small/images/valid',
    
    #train with 1 or 2 classes?
    nc = 1,
    names = ['opacity']
#     nc = 2,
#     names = ['opacity','none']
)

# Note that I am creating the file in the yolov5/data/ directory.
with open('tmp/yolov5/data/data1.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)
    
%cat tmp/yolov5/data/data1.yaml

/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection
{names: [opacity], nc: 1, train: ../covid_small/images/train, val: ../covid_small/images/valid}


In [25]:
%cd tmp/yolov5/
    #train with 1 or 2 classes?
!python train.py --img {IMG_SIZE} \
                 --batch {BATCH_SIZE} \
                 --epochs {EPOCHS} \
                 --data data1.yaml \
                 --weights yolov5s.pt \
                 --save_period 1\
                 --project kaggle-siim-covid_tiny\
                 --single-cls    
    
# !python train.py --img {IMG_SIZE} \
#                  --batch {BATCH_SIZE} \
#                  --epochs {EPOCHS} \
#                  --data data1.yaml \
#                  --weights yolov5s.pt \
#                  --save_period 1\
#                  --project kaggle-siim-covid_tiny
%cd ..

/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection/tmp/yolov5
train: weights=yolov5s.pt, cfg=, data=data1.yaml, hyp=data/hyps/hyp.scratch.yaml, epochs=300, batch_size=64, img_size=[512], rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache_images=False, image_weights=False, device=, multi_scale=False, single_cls=True, adam=False, sync_bn=False, workers=8, project=kaggle-siim-covid_tiny, entity=None, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, upload_dataset=False, bbox_interval=-1, save_period=1, artifact_alias=latest, local_rank=-1
github: Command 'git fetch && git config --get remote.origin.url' timed out after 5 seconds, for updates see https://github.com/ultralytics/yolov5
YOLOv5 🚀 v5.0-294-gdd62e2d torch 1.9.0+cu102 CUDA:0 (TITAN Xp, 12194.0625MB)

hyperparameters: lr0=0.01, lrf=0.2, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5,